## Deliverable 2. Create a Customer Travel Destinations Map.

In [38]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [39]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,victoria,HK,22.2855,114.1577,65.97,90,100,1.01,overcast clouds
1,1,ushuaia,AR,-54.8000,-68.3000,62.26,63,75,11.50,broken clouds
2,2,brae,GB,60.3964,-1.3530,37.31,53,100,32.79,overcast clouds
3,3,tazovskiy,RU,67.4667,78.7000,-25.10,100,89,0.49,overcast clouds
4,4,clyde river,CA,70.4692,-68.5914,-30.77,68,0,10.36,clear sky


In [40]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp=float(input("Input minimum temperature: "))
max_temp=float(input("Input maximum temperature: "))

Input minimum temperature: 70
Input maximum temperature: 90


In [41]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
city_temp_df=city_data_df.loc[(city_data_df["Max Temp"]>=min_temp) & (city_data_df["Max Temp"]<=max_temp)]
city_temp_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
7,7,palmares do sul,BR,-30.2578,-50.5097,70.32,66,22,10.87,few clouds
8,8,codrington,AU,-38.2667,141.9667,83.23,28,0,9.04,clear sky
13,13,natal,BR,-5.7950,-35.2094,81.25,78,20,9.22,few clouds
14,14,port alfred,ZA,-33.5906,26.8910,73.96,78,98,4.90,overcast clouds
15,15,saint-philippe,RE,-21.3585,55.7679,78.39,85,91,13.87,overcast clouds


In [42]:
# 4a. Determine if there are any empty rows.
city_temp_df[city_temp_df.isnull()==True].count()

Unnamed: 0             0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [43]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#none are empty. just reassigning the variable so that it matches below
clean_df=city_temp_df

In [44]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,palmares do sul,BR,70.32,few clouds,-30.2578,-50.5097,
8,codrington,AU,83.23,clear sky,-38.2667,141.9667,
13,natal,BR,81.25,few clouds,-5.7950,-35.2094,
14,port alfred,ZA,73.96,overcast clouds,-33.5906,26.8910,
15,saint-philippe,RE,78.39,overcast clouds,-21.3585,55.7679,
18,hithadhoo,MV,82.31,scattered clouds,-0.6000,73.0833,
20,kedougou,SN,75.78,clear sky,12.5579,-12.1743,
22,ugoofaaru,MV,80.44,broken clouds,5.6667,73.0000,
24,cartagena del chaira,CO,74.34,overcast clouds,1.3349,-74.8429,
25,kalmunai,LK,75.16,scattered clouds,7.4167,81.8167,


In [50]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat=row["Lat"]
    lng=row["Lng"]
    params["location"]=f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels=requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"]=hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found. Skipping...")

        

Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...


In [66]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.drop(hotel_df.index[(hotel_df["Hotel Name"].isnull()==True)], axis=0, inplace=False)
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,palmares do sul,BR,70.32,few clouds,-30.2578,-50.5097,Hotel Campos Dourados.
8,codrington,AU,83.23,clear sky,-38.2667,141.9667,Codrington Gardens Bed And Breakfast
13,natal,BR,81.25,few clouds,-5.7950,-35.2094,Villa Park Hotel | Natal
14,port alfred,ZA,73.96,overcast clouds,-33.5906,26.8910,The Halyards Hotel
15,saint-philippe,RE,78.39,overcast clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""


In [67]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [68]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [69]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))